In [1]:
%config InlineBackend.figure_format = 'retina'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from pathlib import Path
import random
from aging.behavior.syllables import relabel_by_usage
from aging.plotting import figure, save_factory, PlotConfig, format_plots, add_identity, legend
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
format_plots()
c = PlotConfig()

In [3]:
import numpy as np

import scipy
from scipy import stats

import pandas as pd

import matplotlib.pyplot as plt

import matplotlib as mpl
import matplotlib.pyplot as plt
# %matplotlib inline
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from umap import UMAP
from sklearn.metrics import pairwise_distances

from pathlib import Path

In [4]:
df_epi = pd.read_parquet('/n/groups/datta/David/behavior/longtogeny/processed/df_meta_with_epi_age_prediction_72_samples.parquet')

In [5]:
## create mouse map
nage=6 #number of age groups
nmice=16
keep_mice = [
    "_01_01",
    "_01_02",
    "_01_03",
    "_01_04",
    "_02_01",
    "_02_02",
    "_02_03",
    "_02_04",
    "_03_01",
    "_03_02",
    "_03_03",
    "_03_04",
    "_04_01",
    "_04_02",
    "_04_03",
    "_04_04",
] * nage
ages = np.repeat(['4','8','12','52','78','90'],nmice)
mouse_id = [x + y for x, y in zip(ages, keep_mice)]
samples = list(range(1,nage*nmice+1))
mapping = {key: value for key, value in zip(samples, mouse_id)}

In [7]:
df_epi

,sample_rep_id,sample_id,Sentrix_Barcode,Sample_Section,level_0,Sample_Well,age,age_cat,is_tech_rep1,has_rep,is_tech_rep2,outs,weeks,outs_all,array,new_age,all_age,weeks2,ePC1
array_index,,,,,,,,,,,,,,,,,,,
207723860044_R01C01,1,1,207723860044,R01C01,Datta_Project_001,A01,4,4,False,False,False,5.344674,21.378696,5.344674,207723860044,5.344672,4.414918,17.659671,-13.925639
207723860044_R02C01,7,7,207723860044,R02C01,Datta_Project_001,B01,4,4,False,False,False,5.953195,23.812778,5.953195,207723860044,5.953193,4.030478,16.121912,-14.423956
207723860044_R03C01,9,9,207723860044,R03C01,Datta_Project_001,C01,4,4,False,True,True,4.730253,18.921012,4.730253,207723860044,4.730251,3.842482,15.369927,-15.080255
207723860044_R04C01,15,15,207723860044,R04C01,Datta_Project_001,D01,4,4,False,False,False,6.057601,24.230402,6.057601,207723860044,6.057599,4.906493,19.625973,-14.197741
208125290074_R03C01,3,3,208125290074,R03C01,Datta_Project_002,C01,4,4,False,False,False,7.274232,29.096929,7.274232,208125290074,7.274231,3.704482,14.817930,-13.729686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208125290079_R05C02,81B,81,208125290079,R05C02,Datta_Project_002,C05,90,90,True,True,False,18.546283,74.185133,18.546283,208125290079,18.546281,16.348475,65.393899,14.272679
208125290074_R04C02,83,83,208125290074,R04C02,Datta_Project_002,B02,90,90,False,False,False,20.170363,80.681452,20.170363,208125290074,20.170361,17.807456,71.229825,8.054966
208125290080_R06C02,88,88,208125290080,R06C02,Datta_Project_002,H03,90,90,False,False,False,18.058731,72.234925,18.058731,208125290080,18.058730,16.489248,65.956993,11.534314


In [8]:
## add mouse identification to df_meta_merge
df_epi['mouse_id'] = df_epi['sample_id']
df_epi['mouse_id'] = df_epi['mouse_id'].map(mapping)

In [7]:
df_bio = pd.read_parquet('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/ontogeny-bio-age-predictions.parquet')

In [11]:
#filter to find ages in age range of the epidata
fdata = df_bio.reset_index()
fdata['mouse']=fdata['mouse'].str[-6:]
df_bio['mouse_id'] = [x + y for x, y in zip(fdata['age'].astype(str), fdata['mouse'])]

In [12]:
ids = df_epi.mouse_id.unique()
df_bio = df_bio[df_bio.mouse_id.isin(ids)]

In [13]:
# get dataframe of epi data with uuids of beahvioral data
temp = pd.merge(df_epi, df_bio, on='mouse_id', how='inner') 

In [14]:
temp.to_parquet('/n/groups/datta/win/longtogeny/data/epigenetics/2005_df_epi_with_bio_age.parquet')

/home/drl21/miniconda/envs/aging/lib/python3.10/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
